# Chapter 8 連立方程式

## 8.1 線形関数とアフィン関数

### 行列ベクトル積の関数

行列ベクトル積の関数を定義し，重ね合わせの性質を数値的に確認してみよう．

In [1]:
A = [-0.1 2.8 -1.6; 2.3 -0.6 -3.6] # 2x3行列Aの定義

2×3 Array{Float64,2}:
 -0.1   2.8  -1.6
  2.3  -0.6  -3.6

In [2]:
f(x) = A * x # 行列ベクトル積関数の定義

f (generic function with 1 method)

In [3]:
# 線形性をチェック
x = [1, 2, 3]; 
y = [-3, -1, 2];
alpha = 0.5;
beta = -1.6;

In [4]:
lhs = f(alpha * x + beta * y)

2-element Array{Float64,1}:
  9.469999999999999
 16.75             

In [5]:
rhs = alpha * f(x) + beta * f(y)

2-element Array{Float64,1}:
  9.47
 16.75

In [6]:
using LinearAlgebra

In [7]:
norm(lhs - rhs)

1.7763568394002505e-15

In [8]:
f([0, 1, 0])  # Aの第2列のはず

2-element Array{Float64,1}:
  2.8
 -0.6

In [ ]:
### 平均除去行列

平均除去行列を作り，ベクトルに適用してみよう．

In [9]:
using VMLS

In [10]:
de_mean(n) = eye(n) .- 1/n;  # 平均除去行列
x = [0.2, 2.3, 1.0];
de_mean(length(x)) * x  # 行列積で平均除去

3-element Array{Float64,1}:
 -0.9666666666666666 
  1.1333333333333335 
 -0.16666666666666652

In [11]:
x .- avg(x)  # 平均を引いて平均除去

3-element Array{Float64,1}:
 -0.9666666666666668 
  1.133333333333333  
 -0.16666666666666674

### 非線形関数の例

要素毎の絶対値を取る関数やソート関数は非線形関数の例である．これらの関数は`abs`や`sort`で簡単に実現できる．デフォルトでは`sort`関数は昇順にソートするが，キーワード引数で変更できる．


In [12]:
f(x) = abs.(x)   # 要素毎の絶対値

f (generic function with 1 method)

In [13]:
x = [1, 0]; 
y = [0, 1]; 
alpha = -1; 
beta = 2;

In [14]:
f(alpha * x + beta * y)

2-element Array{Int64,1}:
 1
 2

In [15]:
alpha * f(x) + beta * f(y)

2-element Array{Int64,1}:
 -1
  2

In [16]:
f(x) = sort(x, rev = true)  # 降順にソート

f (generic function with 1 method)

In [17]:
f(alpha * x + beta * y)

2-element Array{Int64,1}:
  2
 -1

In [18]:
alpha * f(x) + beta * f(y)

2-element Array{Int64,1}:
 1
 0

## 8.2 線形関数モデル

### 需要の価格弾力性

価格弾力性行列を使って，3つのプロダクトの価格が少し変動した場合の需要を予測しよう．これで，ある製造コストのときの，総利益の変化を予測できる．


In [20]:
p = [10, 20, 15]; # 現在の価格
d = [5.6, 1.5, 8.6]; # 現在の需要（千単位）
c = [6.5, 11.2, 9.8]; # 製造コスト

In [21]:
profit = (p - c)' * d # 現在の総利益

77.51999999999998

In [22]:
# 弾力性行列
E = [-0.3 0.1 -0.1; 0.1 -0.5 0.05 ; -0.1 0.05 -0.4]

3×3 Array{Float64,2}:
 -0.3   0.1   -0.1 
  0.1  -0.5    0.05
 -0.1   0.05  -0.4 

In [24]:
p_new = [9, 21, 14];  # 新しい価格

In [25]:
delta_p = (p_new - p) ./ p  # 価格の変化割合

3-element Array{Float64,1}:
 -0.1                
  0.05               
 -0.06666666666666667

In [26]:
delta_d = E * delta_p  # 予測された需要の変化割合

3-element Array{Float64,1}:
  0.04166666666666667
 -0.03833333333333334
  0.03916666666666667

In [27]:
d_new = d .* (1 .+ delta_d)  # 予測された新しい需要

3-element Array{Float64,1}:
 5.833333333333333
 1.4425           
 8.936833333333333

In [28]:
profit_new = (p_new - c)' * d_new # 予測された新しい利益

66.25453333333333


この価格弾力性の線形モデルを信用するなら，価格を変更しないほうがいい，という結果になった．


### テーラー近似

以下の非線形関数$f: \boldsymbol{R}^2 \to \boldsymbol{R}^2$を考える．
$$
f(x) = \begin{bmatrix}
\| x - a \| \\
\| x - b \| 
\end{bmatrix} = 
\begin{bmatrix}
\sqrt{(x_1 - a_1)^2 + (x_2 - a_2)^2} \\
\sqrt{(x_1 - b_1)^2 + (x_2 - b_2)^2} 
\end{bmatrix}
$$

$f$の2つの要素は$x$から$a$と$b$までの距離である．この関数は$x=a$と$x=b$以外で微分可能である．$f$の導関数，つまりヤコビ行列は次式になる．
$$
D f(x) =
\begin{bmatrix}
\displaystyle\frac{\partial f_1}{\partial x_1}(z) & 
\displaystyle\frac{\partial f_1}{\partial x_2}(z) \\
\displaystyle\frac{\partial f_2}{\partial x_1}(z) & 
\displaystyle\frac{\partial f_2}{\partial x_2}(z)
\end{bmatrix} =
\begin{bmatrix}
\displaystyle\frac{z_1 - a_1}{\| z - a \|} & 
\displaystyle\frac{z_2 - a_2}{\| z - a \|}\\
\displaystyle\frac{z_1 - b_1}{\| z - b \|} & 
\displaystyle\frac{z_2 - b_2}{\| z - b \|}
\end{bmatrix}
$$

では$a, b, z$を与えて$f$をテーラー近似し，$z$周辺の点で真の$f$の値と比較してみよう．


In [29]:
f(x) = [ norm(x - a), norm(x - b) ];
Df(z) = [ (z - a)' / norm(z - a) ; (z - b)' / norm(z - b) ];
f_hat(x) = f(z) + Df(z) * (x - z);
a = [1, 0]; 
b = [1, 1];
z = [0, 0];

In [30]:
f([0.1, 0.1])

2-element Array{Float64,1}:
 0.9055385138137417
 1.2727922061357855

In [31]:
f_hat([0.1, 0.1])

2-element Array{Float64,1}:
 0.9               
 1.2727922061357857

In [32]:
f([0.5, 0.5])

2-element Array{Float64,1}:
 0.7071067811865476
 0.7071067811865476

In [33]:
f_hat([0.5, 0.5])

2-element Array{Float64,1}:
 0.5               
 0.7071067811865477

### 回帰モデル

2.3節の家の売却価格データのための回帰モデル
$$
\hat{y} = x^T \beta + v = \beta_1 x_1 + \beta_2 x_2 + v
$$
をもう一度見てみよう．ここで$\hat{y}$は予測された売却価格，$x_1$は1000平方フィート単位での家の面積，$x_2$はベッドルーム数である．

以下のコードでは，$2 \times 774$のデータ行列$X$，結果ベクトル$y^d$を作る．データセット中のサンプル数は$N=774$である．そして回帰モデルの予測$\hat{y}^d$，予測誤差$r^d$，RMS予測誤差を計算する．

In [34]:
# 回帰モデルのパラメータ
beta = [148.73, -18.85]; 
v = 54.40;
D = house_sales_data();
yd = D["price"]; # 結果ベクトル

In [35]:
N = length(yd)

774

In [36]:
X = [ D["area"] D["beds"] ]';

In [37]:
size(X)

(2, 774)

In [38]:
ydhat = X' * beta .+ v; # 予測結果ベクトル

In [39]:
rd = yd - ydhat; # 予測誤差ベクトル

In [40]:
rms(rd) # RMS予測誤差

74.84571862623022

In [41]:
# 価格の標準偏差と比較
stdev(yd)

112.78216159756509


## 8.3 連立方程式

### 化学反応のBalancing

本書＊＊＊ページの化学平衡の方程式を
水の電気分解という単純な例で検証してみる．


In [42]:
R = [2 ; 1]

2-element Array{Int64,1}:
 2
 1

In [43]:
P = [2 0 ; 0 2]

2×2 Array{Int64,2}:
 2  0
 0  2

In [44]:
# 反応係数 [2,2,1]で確認
coeff = [2, 2, 1];

In [46]:
[R -P] * coeff

2-element Array{Int64,1}:
 0
 0